In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
import numpy as np

print(f"tensorflow {tf.__version__}")
print(f"Numpy {np.__version__}")

tensorflow 2.5.0
Numpy 1.19.2


![VAE](VAE.png)

In [4]:
class Sampling(tf.keras.layers.Layer):
	def __init__(self):
		super(Sampling, self).__init__()
	def call(self, inputs):
		z_mean,z_log_var = inputs
		print(z_mean)
		batch_data = tf.shape(z_mean)[0]
		dim_data = tf.shape(z_mean)[1]
		epsilon = K.random_normal(shape= (batch_data, dim_data), mean=0., stddev=1.0)
		return z_mean + tf.exp(z_log_var * 0.5) * epsilon


In [16]:

BATCH_SIZE = 64,
ORIGINAL_DIM = 28 * 28

LATENT_DIM = 2 # tHE cOMPRESSED dIMENSION/ Bottleneck
INTERMEDIATE_DIM = 256 # The units For Decoder and Encoder
EPOCHS = 5 #  Total Epochs
epsilon_std = 1.0 # For generating The Gaussian 

### E N C O D E R ####
x = Input(shape=(28,28,1), name="Input_Images") # input Layers , just Pass the Original Dims , Vector with 784 elements
h = Dense(INTERMEDIATE_DIM,activation=tf.nn.relu, name="Encoding")(x) # This Layers is For encoding the Input
h = tf.keras.layers.Flatten()(h) # This Layers is For Flattening the Input
z_mean = Dense(LATENT_DIM, name="Mean")(h) # This layer Function is For Getting the Mean of the Gaussian that we're going to create
z_log_var = Dense(LATENT_DIM, name="Log_Variance")(h)  # This Layer is for getting the variance of the Gaussian but the LOG , its same as the variance but more stable
z = Sampling()([z_mean, z_log_var]) # This is the Sampling Function , this will be used for getting the latent vector
encoder = Model(x, [z_mean, z_log_var,z], name="Encoder")  # Creating the models for the Encoder
print(encoder.summary())
### E N C O D E R ####
#### D E C O D E R ####
input_decoder = Input(shape=(LATENT_DIM,), name='decoder_input') # This Is Input for decoder
decoder_h = Dense(INTERMEDIATE_DIM, activation=tf.nn.relu,name="decoder_h")(input_decoder) # This is The layers for decoder
input_decoded = Dense(ORIGINAL_DIM,activation=tf.nn.sigmoid,name="Flattened_decoded")(decoder_h) # This layers is for The output < Reconstruted input > 
reshaped = tf.reshape(input_decoded, (-1,28,28,1)) # Reshape the input , -1 For Batch
decoder = Model(input_decoder,reshaped,name="decoder") # Creating the Decoder Models
print(decoder.summary())
## Sampling / Reparameterization trick

####  Creating Variational AutoEncoder ####
#print(create_decoder().summary())
#### D E C O D E R ####
class VAE(tf.keras.Model):
	def __init__ (self,encoder,decoder,**kwargs):
		super(VAE,self).__init__()
		self.encoder = encoder
		self.decoder = decoder

	def train_step(self,data):
		if isinstance(data, tuple):
			data = data[0]
		with tf.GradientTape() as tape:
			z_mean,z_log_var,z = self.encoder(data)
			
			reconstruction  = self.decoder(z)
			reconstruction_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(data,reconstruction))
			reconstruction_loss *= ORIGINAL_DIM
			
			kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
			kl_loss = tf.reduce_sum(kl_loss,axis=None)
			kl_loss *= -0.5

			total_loss = reconstruction_loss + kl_loss
		grads = tape.gradient(total_loss, self.trainable_variables)

		self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
		return {
			"loss" : total_loss,
			"reconstruction_loss" : reconstruction_loss,
			"kl_loss" : kl_loss
		}
(x_train, _), (x_test, _) = tf.keras.datasets.fashion_mnist.load_data()
fmnist_images = np.concatenate([x_train, x_test], axis = 0)
# expand dimension to add  a color map dimension
fmnist_images = np.expand_dims(fmnist_images, -1).astype("float32") / 255
  
# compile and train the model
vae = VAE(encoder, decoder)
vae.compile(optimizer ='rmsprop')
vae.fit(fmnist_images, epochs = 100, batch_size = 64)

Tensor("Placeholder:0", shape=(None, 2), dtype=float32)
Model: "Encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Images (InputLayer)       [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
Encoding (Dense)                (None, 28, 28, 256)  512         Input_Images[0][0]               
__________________________________________________________________________________________________
flatten_4 (Flatten)             (None, 200704)       0           Encoding[0][0]                   
__________________________________________________________________________________________________
Mean (Dense)                    (None, 2)            401410      flatten_4[0][0]                  
____________________________________

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [ ]:
vae.fit(x_train, x_train,
        shuffle=True,
        epochs=EPOCHS,)

ValueError: ('Error when checking model target: expected no data, but got:', array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))